# Zillow Property Search
[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/drive/1afLUQVYLIhv0vDB-GNuYgxfZrJhTzSM5?usp=sharing)

## Overview
| Detail Tag            | Information                                                                                        |
|-----------------------|----------------------------------------------------------------------------------------------------|
| Originally Created By | Ariel Herrera arielherrera@analyticsariel.com |
| External References   | API |
| Input Datasets        | Source name |
| Output Datasets       | Source name |
| Input Data Source     | Pandas DataFrame |
| Output Data Source    | Pandas DataFrame |

## History
| Date         | Developed By  | Reason                                                |
|--------------|---------------|-------------------------------------------------------|
| 22nd Jan 2022 | Ariel Herrera | Create notebook. |

## Getting Started
1. Copy this notebook -> File -> Save a Copy in Drive
2. Directions

## Useful Resources
- [Google Colab Cheat Sheet](https://towardsdatascience.com/cheat-sheet-for-google-colab-63853778c093)

## <font color="blue">Install Packages</font>

## <font color="blue">Imports</font>

In [1]:
from google.colab import output, drive, files # specific to Google Colab
import pandas as pd
import numpy as np
import plotly.express as px
import requests
import warnings

# settings
warnings.filterwarnings("ignore")
pd.set_option("display.max_columns", None)

## <font color="blue">Functions</font>

In [2]:
def get_listings(api_key, listing_url):
    url = "https://app.scrapeak.com/v1/scrapers/zillow/listing"

    querystring = {
        "api_key": api_key,
        "url":listing_url
    }

    return requests.request("GET", url, params=querystring)

def get_property_detail(api_key, zpid):
    url = "https://app.scrapeak.com/v1/scrapers/zillow/property"

    querystring = {
        "api_key": api_key,
        "zpid":zpid
    }

    return requests.request("GET", url, params=querystring)

def get_zpid(api_key, street, city, state, zip_code=None):
    url = "https://app.scrapeak.com/v1/scrapers/zillow/zpidByAddress"

    querystring = {
        "api_key": api_key,
        "street": street,
        "city": city,
        "state": state,
        "zip_code":zip_code
    }

    return requests.request("GET", url, params=querystring)

## <font color="blue">Locals & Constants</font>

In [3]:
############
# OPTIONAL #
############

# mount drive
drive.mount("/content/drive", force_remount=False)

# data location
file_dir = "/content/drive/My Drive/Colab Data/input/" # optional

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
# read in api key file
df_api_keys = pd.read_csv(file_dir + "api_keys.csv")

# get keys
api_key = df_api_keys.loc[df_api_keys["API"] == "scrapeak"]["KEY"].iloc[0] # replace this with your own key

## <font color="blue">Data</font>

### <font color="green">1. Property Listings For Sale</font>

In [5]:
# zillow search url
listing_url = "https://www.zillow.com/tampa-fl/houses/?searchQueryState=%7B%22pagination%22%3A%7B%7D%2C%22usersSearchTerm%22%3A%22Tampa%2C%20FL%22%2C%22mapBounds%22%3A%7B%22west%22%3A-82.96171153027345%2C%22east%22%3A-81.94684946972657%2C%22south%22%3A27.709897826482354%2C%22north%22%3A28.2786076346999%7D%2C%22regionSelection%22%3A%5B%7B%22regionId%22%3A41176%2C%22regionType%22%3A6%7D%5D%2C%22isMapVisible%22%3Atrue%2C%22filterState%22%3A%7B%22price%22%3A%7B%22min%22%3A200000%2C%22max%22%3A550000%7D%2C%22mp%22%3A%7B%22min%22%3A946%2C%22max%22%3A2366%7D%2C%22beds%22%3A%7B%22min%22%3A2%7D%2C%22built%22%3A%7B%22min%22%3A1940%7D%2C%22doz%22%3A%7B%22value%22%3A%2290%22%7D%2C%22sort%22%3A%7B%22value%22%3A%22globalrelevanceex%22%7D%2C%22ah%22%3A%7B%22value%22%3Atrue%7D%2C%22con%22%3A%7B%22value%22%3Afalse%7D%2C%22mf%22%3A%7B%22value%22%3Afalse%7D%2C%22manu%22%3A%7B%22value%22%3Afalse%7D%2C%22land%22%3A%7B%22value%22%3Afalse%7D%2C%22tow%22%3A%7B%22value%22%3Afalse%7D%2C%22apa%22%3A%7B%22value%22%3Afalse%7D%2C%22apco%22%3A%7B%22value%22%3Afalse%7D%7D%2C%22isListVisible%22%3Atrue%7D"

# get listings
listing_response = get_listings(api_key, listing_url)

In [6]:
# view all keys
listing_response.json().keys()

dict_keys(['is_success', 'data', 'message'])

In [7]:
# check if request is successful
listing_response.json()["is_success"]

True

In [8]:
# view count of properies returned in request
num_of_properties = listing_response.json()["data"]["categoryTotals"]["cat1"]["totalResultCount"]
print("Count of properties:", num_of_properties)

Count of properties: 219


In [9]:
# view all listings
df_listings = pd.json_normalize(listing_response.json()["data"]["cat1"]["searchResults"]["mapResults"])
print("Number of rows:", len(df_listings))
print("Number of columns:", len(df_listings.columns))
df_listings

Number of rows: 211
Number of columns: 70


,zpid,price,priceLabel,beds,baths,area,statusType,statusText,isFavorite,isUserClaimingOwner,isUserConfirmedClaim,imgSrc,hasImage,visited,listingType,shouldShowZestimateAsPrice,detailUrl,pgapt,sgapt,has3DModel,hasVideo,isHomeRec,address,hasAdditionalAttributions,isFeaturedListing,availabilityDate,latLong.latitude,latLong.longitude,variableData.type,variableData.text,hdpData.homeInfo.zpid,hdpData.homeInfo.zipcode,hdpData.homeInfo.city,hdpData.homeInfo.state,hdpData.homeInfo.latitude,hdpData.homeInfo.longitude,hdpData.homeInfo.price,hdpData.homeInfo.bathrooms,hdpData.homeInfo.bedrooms,hdpData.homeInfo.livingArea,hdpData.homeInfo.homeType,hdpData.homeInfo.homeStatus,hdpData.homeInfo.daysOnZillow,hdpData.homeInfo.isFeatured,hdpData.homeInfo.shouldHighlight,hdpData.homeInfo.zestimate,hdpData.homeInfo.rentZestimate,hdpData.homeInfo.listing_sub_type.is_FSBA,hdpData.homeInfo.isUnmappable,hdpData.homeInfo.isPreforeclosureAuction,hdpData.homeInfo.homeStatusForHDP,hdpData.homeInfo.priceForHDP,hdpData.homeInfo.isNonOwnerOccupied,hdpData.homeInfo.isPremierBuilder,hdpData.homeInfo.isZillowOwned,hdpData.homeInfo.currency,hdpData.homeInfo.country,hdpData.homeInfo.taxAssessedValue,hdpData.homeInfo.lotAreaValue,hdpData.homeInfo.lotAreaUnit,variableData,hdpData.homeInfo.listing_sub_type.is_newHome,hdpData.homeInfo.newConstructionType,hdpData.homeInfo.listing_sub_type.is_openHouse,hdpData.homeInfo.openHouse,hdpData.homeInfo.open_house_info.open_house_showing,hdpData.homeInfo.datePriceChanged,hdpData.homeInfo.priceReduction,hdpData.homeInfo.priceChange,hdpData.homeInfo.unit
0,45033671,"$285,000",$285K,2,1.0,929,FOR_SALE,House for sale,False,False,False,https://photos.zillowstatic.com/fp/f8f1113a921...,True,False,,False,/homedetails/10414-Cliff-Cir-Tampa-FL-33612/45...,ForSale,For Sale (Broker),True,False,False,--,False,False,None,28.045662,-82.467896,3D_HOME,3D Tour,45033671,33612,Tampa,FL,28.045662,-82.467896,285000.0,1.0,2.0,929.0,SINGLE_FAMILY,FOR_SALE,-1,False,False,287000.0,1494.0,True,False,False,FOR_SALE,285000.0,True,False,False,USD,USA,124794.0,7625.0,sqft,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,82046466,"$265,000",$265K,3,2.0,1290,FOR_SALE,New construction,False,False,False,https://photos.zillowstatic.com/fp/813010716aa...,True,False,NEW_CONSTRUCTION,False,/homedetails/8206-N-14th-St-Tampa-FL-33604/820...,ForSale,New Construction,False,False,False,--,False,False,None,28.022974,-82.444450,NaN,NaN,82046466,33604,Tampa,FL,28.022974,-82.444450,265000.0,2.0,3.0,1290.0,SINGLE_FAMILY,FOR_SALE,-1,False,False,NaN,NaN,NaN,False,False,FOR_SALE,265000.0,True,False,False,USD,USA,21250.0,5000.0,sqft,NaN,True,BUILDER_SPEC,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,45072398,"$444,000",$444K,3,2.0,1413,FOR_SALE,House for sale,False,False,False,https://photos.zillowstatic.com/fp/752a561c952...,True,False,,False,/homedetails/7019-S-West-Shore-Blvd-Tampa-FL-3...,ForSale,For Sale (Broker),False,False,False,--,False,False,None,27.863548,-82.527200,OPEN_HOUSE,Open: Sat. 11am-2pm,45072398,33616,Tampa,FL,27.863548,-82.527200,444000.0,2.0,3.0,1413.0,SINGLE_FAMILY,FOR_SALE,-1,False,False,375200.0,2806.0,True,False,False,FOR_SALE,444000.0,True,False,False,USD,USA,195590.0,4750.0,sqft,NaN,NaN,NaN,True,Sat. 11am-2pm,"[{'open_house_start': 1677351600000, 'open_hou...",NaN,NaN,NaN,NaN
3,44894194,"$275,000",$275K,3,2.0,1596,FOR_SALE,House for sale,False,False,False,https://photos.zillowstatic.com/fp/18032010d62...,True,False,,False,/homedetails/122-W-138th-Ave-Tampa-FL-33613/44...,ForSale,For Sale (Broker),False,False,False,--,False,False,None,28.072765,-82.461110,NaN,NaN,44894194,33613,Tampa,FL,28.072765,-82.461110,275000.0,2.0,3.0,1596.0,SINGLE_FAMILY,FOR_SALE,-1,False,False,NaN,NaN,True,False,False,FOR_SALE,275000.0,True,False,False,USD,USA,151683.0,7900.0,sqft,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2068859994,"$525,000",$525K,3,2.0,1368,FOR_SALE,House for sale,False,False,False,https://photos.zillowstatic.com/fp/577f30f70a5...,True,False,,False,/homedetails/2817-W-Leila-Ave-

In [10]:
# view price
px.histogram(df_listings, x="hdpData.homeInfo.price", title="Sales Price Histogram")

In [11]:
# view zestimate
px.histogram(df_listings, x="hdpData.homeInfo.zestimate", title="Zestimate Histogram")

In [12]:
# view rent zestimate
px.box(df_listings, x="hdpData.homeInfo.rentZestimate", title="Rent Zestimate Box Plot")

### <font color="green">2. Rental Property Listings For Rent</font>

In [13]:
# zillow search url
rent_listing_url = "https://www.zillow.com/tampa-fl/rentals/?searchQueryState=%7B%22pagination%22%3A%7B%7D%2C%22usersSearchTerm%22%3A%22Tampa%2C%20FL%22%2C%22mapBounds%22%3A%7B%22west%22%3A-82.69632304150392%2C%22east%22%3A-82.2122379584961%2C%22south%22%3A27.743327003864252%2C%22north%22%3A28.2453440906427%7D%2C%22regionSelection%22%3A%5B%7B%22regionId%22%3A41176%2C%22regionType%22%3A6%7D%5D%2C%22isMapVisible%22%3Atrue%2C%22filterState%22%3A%7B%22sort%22%3A%7B%22value%22%3A%22priorityscore%22%7D%2C%22fsba%22%3A%7B%22value%22%3Afalse%7D%2C%22fsbo%22%3A%7B%22value%22%3Afalse%7D%2C%22nc%22%3A%7B%22value%22%3Afalse%7D%2C%22cmsn%22%3A%7B%22value%22%3Afalse%7D%2C%22auc%22%3A%7B%22value%22%3Afalse%7D%2C%22fore%22%3A%7B%22value%22%3Afalse%7D%2C%22pmf%22%3A%7B%22value%22%3Afalse%7D%2C%22pf%22%3A%7B%22value%22%3Afalse%7D%2C%22fr%22%3A%7B%22value%22%3Atrue%7D%2C%22beds%22%3A%7B%22min%22%3A3%2C%22max%22%3A3%7D%2C%22baths%22%3A%7B%22min%22%3A2%7D%2C%22apco%22%3A%7B%22value%22%3Afalse%7D%2C%22apa%22%3A%7B%22value%22%3Afalse%7D%2C%22con%22%3A%7B%22value%22%3Afalse%7D%2C%22tow%22%3A%7B%22value%22%3Afalse%7D%7D%2C%22isListVisible%22%3Atrue%2C%22mapZoom%22%3A11%7D"

# get listings
rent_listing_response = get_listings(api_key, rent_listing_url)

In [14]:
# view all keys
print(rent_listing_response.json().keys())

# check if request is successful
print("Request success:", rent_listing_response.json()["is_success"])

# view count of properies returned in request
num_of_properties = rent_listing_response.json()["data"]["categoryTotals"]["cat1"]["totalResultCount"]
print("Count of properties:", num_of_properties)

dict_keys(['is_success', 'data', 'message'])
Request success: True
Count of properties: 186


In [15]:
# view all listings
df_rent_listings = pd.json_normalize(rent_listing_response.json()["data"]["cat1"]["searchResults"]["mapResults"])
print("Number of rows:", len(df_rent_listings))
print("Number of columns:", len(df_rent_listings.columns))
df_rent_listings

Number of rows: 181
Number of columns: 77


,zpid,price,priceLabel,beds,baths,area,statusType,statusText,isFavorite,isUserClaimingOwner,isUserConfirmedClaim,imgSrc,hasImage,visited,listingType,shouldShowZestimateAsPrice,detailUrl,pgapt,sgapt,has3DModel,hasVideo,isHomeRec,address,hasAdditionalAttributions,isFeaturedListing,availabilityDate,latLong.latitude,latLong.longitude,variableData.type,variableData.text,variableData.data.isFresh,hdpData.homeInfo.zpid,hdpData.homeInfo.zipcode,hdpData.homeInfo.city,hdpData.homeInfo.state,hdpData.homeInfo.latitude,hdpData.homeInfo.longitude,hdpData.homeInfo.price,hdpData.homeInfo.bathrooms,hdpData.homeInfo.bedrooms,hdpData.homeInfo.livingArea,hdpData.homeInfo.homeType,hdpData.homeInfo.homeStatus,hdpData.homeInfo.daysOnZillow,hdpData.homeInfo.isFeatured,hdpData.homeInfo.shouldHighlight,hdpData.homeInfo.isRentalWithBasePrice,hdpData.homeInfo.zestimate,hdpData.homeInfo.rentZestimate,hdpData.homeInfo.isUnmappable,hdpData.homeInfo.isPreforeclosureAuction,hdpData.homeInfo.homeStatusForHDP,hdpData.homeInfo.priceForHDP,hdpData.homeInfo.isNonOwnerOccupied,hdpData.homeInfo.isPremierBuilder,hdpData.homeInfo.isZillowOwned,hdpData.homeInfo.currency,hdpData.homeInfo.country,hdpData.homeInfo.taxAssessedValue,hdpData.homeInfo.datePriceChanged,hdpData.homeInfo.priceReduction,hdpData.homeInfo.priceChange,hdpData.homeInfo.lotAreaValue,hdpData.homeInfo.lotAreaUnit,hdpData.homeInfo.unit,streetViewMetadataURL,streetViewURL,buildingId,lotId,minBeds,minBaths,minArea,plid,unitCount,isBuilding,badgeInfo,canSaveBuilding
0,45050855,"$7,850/mo",$7.9K,3.0,2.5,2093.0,FOR_RENT,House for rent,False,False,False,https://photos.zillowstatic.com/fp/e8d96cc0a17...,True,False,,False,/homedetails/2525-W-Edgewood-Rd-Tampa-FL-33609...,ForRent,For Rent,False,False,False,--,False,True,2023-02-23 00:00:00,27.935354,-82.48765,TIME_ON_INFO,2 hours ago,True,4.505086e+07,33609,Tampa,FL,27.935354,-82.48765,7850.0,2.5,3.0,2093.0,SINGLE_FAMILY,FOR_RENT,-1.0,True,False,False,1815100.0,10311.0,False,False,FOR_RENT,7850.0,True,False,False,USD,USA,876431.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,44922044,"$3,829/mo",$3.8K,3.0,2.0,1678.0,FOR_RENT,House for rent,False,False,False,https://photos.zillowstatic.com/fp/56238f0519f...,True,False,,False,/homedetails/18134-Antietam-Ct-Tampa-FL-33647/...,ForRent,For Rent,True,False,False,--,False,True,None,28.135750,-82.36906,3D_HOME,3D Tour,NaN,4.492204e+07,33647,Tampa,FL,28.135750,-82.36906,3829.0,2.0,3.0,1678.0,SINGLE_FAMILY,FOR_RENT,-1.0,True,False,False,425000.0,2499.0,False,False,FOR_RENT,3829.0,True,False,False,USD,USA,248999.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,45097707,"$2,295/mo",$2.3K,3.0,2.0,1000.0,FOR_RENT,House for rent,False,False,False,https://photos.zillowstatic.com/fp/2a4faaa1463...,True,False,,False,/homedetails/1416-E-Mohawk-Ave-Tampa-FL-33604/...,ForRent,For Rent,False,False,False,--,False,True,2023-02-27 00:00:00,27.997164,-82.44421,TIME_ON_INFO,1 day ago,False,4.509771e+07,33604,Tampa,FL,27.997164,-82.44421,2295.0,2.0,3.0,1000.0,SINGLE_FAMILY,FOR_RENT,-1.0,True,False,False,289600.0,1999.0,False,False,FOR_RENT,2295.0,True,False,False,USD,USA,137538.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,45102332,"$2,500/mo",$2.5K,3.0,2.0,1575.0,FOR_RENT,House for rent,False,False,False,https://photos.zillowstatic.com/fp/a8274b60cfa...,True,False,,False,/homedetails/1908-W-Saint-Louis-St-Tampa-FL-33...,ForRent,For Rent,False,False,False,--,False,True,2023-03-01 00:00:00,27.964973,-82.47924,TIME_ON_INFO,14 hours ago,True,4.510233e+07,33607,Tampa,FL,27.964973,-82.47924,2500.0,2.0,3.0,1575.0,SINGLE_FAMILY,FOR_RENT,-1.0,True,False,False,440600.0,2999.0,False,False,FOR_RENT,2500.0,True,False,False,USD,USA,235420.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,68594382,"$1,950/mo",$2.0K,3.0,2.0,1476.0,FOR_RENT,House for rent,False,False,False,https://photos.zillowstatic.com/fp/490c57e3e8a...,True,False,,False,/homede

In [16]:
# view price
px.box(df_rent_listings, x="hdpData.homeInfo.price", title="Rental Price Box Plot")

In [17]:
# view rent zestimate
px.box(df_rent_listings, x="hdpData.homeInfo.rentZestimate", title="Rent Zestimate Box Plot")

### <font color="green">3. Property Details</font>

In [18]:
# property unique id
zpid = "99616959"

# get property detail
prop_detail_response = get_property_detail(api_key, zpid)

In [19]:
# view all keys
print(prop_detail_response.json().keys())

# check if request is successful
print("Request success:", prop_detail_response.json()["is_success"])

dict_keys(['is_success', 'data', 'message'])
Request success: True


In [20]:
# view property details
df_prop = pd.json_normalize(prop_detail_response.json()['data'])
print("Number of rows:", len(df_prop))
print("Number of columns:", len(df_prop.columns))
df_prop

Number of rows: 1
Number of columns: 602


,listingDataSource,zpid,city,state,homeStatus,isListingClaimedByCurrentSignedInUser,isCurrentSignedInAgentResponsible,bedrooms,bathrooms,price,yearBuilt,streetAddress,zipcode,isCurrentSignedInUserVerifiedOwner,regionString,propertyUpdatePageLink,moveHomeMapLocationLink,propertyEventLogLink,editPropertyHistorylink,isRentalListingOffMarket,hdpUrl,nearbyCities,nearbyNeighborhoods,country,nearbyZipcodes,abbreviatedAddress,building,isUndisclosedAddress,boroughId,providerListingID,boroughSearchUrl,communityUrl,isPremierBuilder,isZillowOwned,homeType,currency,listPriceLow,livingArea,livingAreaValue,zestimate,newConstructionType,zestimateLowPercent,zestimateHighPercent,rentZestimate,restimateLowPercent,restimateHighPercent,schools,homeValues,nearbyHomes,countyFIPS,parcelId,taxHistory,priceHistory,comps,description,whatILove,contingentListingType,timeOnZillow,pageViewCount,favoriteCount,daysOnZillow,latitude,longitude,openHouseSchedule,desktopWebHdpImageLink,brokerageName,timeZone,pals,listingAccountUserId,homeInsights,sellingSoon,listingProvider,isIncomeRestricted,brokerId,ssid,monthlyHoaFee,propertyTaxRate,hiResImageLink,hdpTypeDimension,mlsid,propertyTypeDimension,mediumImageLink,enhancedBrokerImageUrl,responsivePhotos,buildingId,virtualTourUrl,hasApprovedThirdPartyVirtualTourUrl,photoCount,livingAreaUnits,lotSize,lotAreaValue,lotAreaUnits,postingProductType,marketingName,richMedia,cityId,stateId,zipPlusFour,numberOfUnitsTotal,foreclosureDefaultFilingDate,foreclosureAuctionFilingDate,foreclosureLoanDate,foreclosureLoanOriginator,foreclosureLoanAmount,foreclosurePriorSaleDate,foreclosurePriorSaleAmount,foreclosureBalanceReportingDate,foreclosurePastDueBalance,foreclosureUnpaidBalance,foreclosureAuctionTime,foreclosureAuctionDescription,foreclosureAuctionCity,foreclosureAuctionLocation,foreclosureDate,foreclosureAmount,foreclosingBank,foreclosureJudicialType,datePostedString,foreclosureMoreInfo,hasBadGeocode,streetViewMetadataUrlMediaWallLatLong,streetViewMetadataUrlMediaWallAddress,streetViewTileImageUrlMediumLatLong,streetViewTileImageUrlMediumAddress,streetViewServiceUrl,postingUrl,hasPublicVideo,primaryPublicVideo,richMediaVideos,photos,tourViewCount,listingAccount,listingFeedID,livingAreaUnitsShort,priceChange,priceChangeDate,priceChangeDateString,hideZestimate,comingSoonOnMarketDate,isPreforeclosureAuction,lastSoldPrice,isHousingConnector,responsivePhotosOriginalRatio,streetViewMetadataUrlMapLightboxAddress,thumb,isRecentStatusChange,isNonOwnerOccupied,county,isFeatured,rentalApplicationsAcceptedType,listingTypeDimension,featuredListingTypeDimension,brokerIdDimension,keystoneHomeStatus,pageUrlFragment,isRentalsLeadCapMet,isPaidMultiFamilyBrokerId,address.streetAddress,address.city,address.state,address.zipcode,address.neighborhood,address.community,address.subdivision,listing_sub_type.is_newHome,listing_sub_type.is_FSBO,listing_sub_type.is_FSBA,listing_sub_type.is_foreclosure,listing_sub_type.is_bankOwned,listing_sub_type.is_forAuction,listing_sub_type.is_comingSoon,listing_sub_type.is_pending,listing_sub_type.is_openHouse,citySearchUrl.text,citySearchUrl.path,zipcodeSearchUrl.path,apartmentsForRentInZipcodeSearchUrl.path,housesForRentInZipcodeSearchUrl.path,neighborhoodRegion.name,neighborhoodSearchUrl.path,stateSearchUrl.path,countySearchUrl.text,countySearchUrl.path,adTargets.aamgnrc1,adTargets.bd,adTargets.fsbid,adTargets.city,adTargets.proptp,adTargets.pid,adTargets.lot,adTargets.zestibuck,adTargets.listtp,adTargets.sqftrange,adTargets.price,adTargets.sqft,adTargets.state,adTargets.mlong,adTargets.cnty,adTargets.prange,adTargets.zip,adTargets.mlat,adTargets.zusr,adTargets.price_band,adTargets.yrblt,adTargets.zestimate,adTargets.premieragent,adTargets.dma,adTargets.guid,adTargets.ssid,resoFacts.accessibilityFeatures,resoFacts.additionalFeeInfo,resoFacts.associationFee,resoFacts.associationAmenities,resoFacts.associationFee2,resoFacts.associationFeeIncludes,resoFacts.associationName,resoFacts.associationName2,resoFacts.associ

In [21]:
print("Street address:", df_prop["streetAddress"].iloc[0])
print("City:", df_prop["city"].iloc[0])
print("State:", df_prop["state"].iloc[0])
print("Home status:", df_prop["homeStatus"].iloc[0])
print("Bedrooms:", df_prop["bedrooms"].iloc[0])
print("Bathrooms:", df_prop["bathrooms"].iloc[0])
print("Year built:", df_prop["yearBuilt"].iloc[0])
print("Zestimate: ${:,}".format(df_prop["zestimate"].iloc[0]))
print("Rent zestimate: ${:,}".format(df_prop["rentZestimate"].iloc[0]))
print("Description:")
df_prop["description"].iloc[0]

Street address: 3611 Potter St
City: Tampa
State: FL
Home status: FOR_SALE
Bedrooms: 3
Bathrooms: 2
Year built: 2020
Zestimate: $339,553
Rent zestimate: $2,414
Description:


'WOW! What an amazing NEW BUILD with 3 bedrooms, 2 bathrooms, 1 car garage, fully fenced backyard and an incredible kitchen. Fall in love with the gorgeous landscaping in your front yard and a front porch with exterior lighting. As soon as you go inside immediately cherish the openness and TILED great room with your living room to your right, kitchen in the middle including GRANITE COUNTERTOPS, real wood cabinets, WALK-IN PANTRY with wire shelving racks and STAINLESS STEEL APPLIANCES. As you go farther inside, to your left you can find the TILED dining room with BLINDS and a new light fixture. Following the TILE FLOOR into the hallway on the right appreciate your washer and dryer in a closet, tucked away until it’s laundry day. As you keep going through the hallway you can find the CARPETED master bedroom with a WALK IN CLOSET, blinds and on the side is your TILED master bathroom with DUAL SINKS, granite countertops, real wood cabinets, and a stand up shower. As you exit the master bed

#### Comparables

In [22]:
# view comparables
df_c = pd.DataFrame(df_prop["comps"].iloc[0])
df_c

,zpid,miniCardPhotos,price,currency,bedrooms,bathrooms,livingArea,livingAreaValue,livingAreaUnits,livingAreaUnitsShort,listingMetadata,lotSize,lotAreaValue,lotAreaUnits,address,formattedChip,latitude,longitude,homeStatus,homeType,hdpUrl,listing_sub_type,providerListingID,attributionInfo,isPremierBuilder,state,isZillowOwned,newConstructionType
0,45108843,[{'url': 'https://photos.zillowstatic.com/fp/d...,325000,USD,3,1,1064,1064,Square Feet,sqft,{'comminglingCategoryIsRulesApplicable': True},3500,3500.000000,Square Feet,"{'streetAddress': '1801 E 18th Ave', 'city': '...",{'location': [{'fullValue': '1801 E 18th Ave'}...,27.968065,-82.438710,FOR_SALE,SINGLE_FAMILY,/homedetails/1801-E-18th-Ave-Tampa-FL-33605/45...,"{'is_newHome': False, 'is_forAuction': False, ...",None,"{'mlsId': 'T3430530', 'mlsName': None, 'provid...",False,FL,False,None
1,45100317,[{'url': 'https://photos.zillowstatic.com/fp/2...,394995,USD,3,3,1523,1523,Square Feet,sqft,{'comminglingCategoryIsRulesApplicable': True},2450,2450.000000,Square Feet,"{'streetAddress': '3618 N 25th St', 'city': 'T...","{'location': [{'fullValue': '3618 N 25th St'},...",27.977478,-82.431564,FOR_SALE,SINGLE_FAMILY,/homedetails/3618-N-25th-St-Tampa-FL-33605/451...,"{'is_newHome': True, 'is_forAuction': False, '...",None,"{'mlsId': 'T3429831', 'mlsName': None, 'provid...",False,FL,False,BUILDER_SPEC
2,45108732,[{'url': 'https://photos.zillowstatic.com/fp/f...,329999,USD,3,2,1092,1092,Square Feet,sqft,{'comminglingCategoryIsRulesApplicable': True},30491,0.700000,Acres,"{'streetAddress': '2904 Sanchez St', 'city': '...",{'location': [{'fullValue': '2904 Sanchez St'}...,27.968900,-82.442340,FOR_SALE,SINGLE_FAMILY,/homedetails/2904-Sanchez-St-Tampa-FL-33605/45...,"{'is_newHome': False, 'is_forAuction': False, ...",None,"{'mlsId': '821464', 'mlsName': None, 'provider...",False,FL,False,None
3,45099808,[{'url': 'https://photos.zillowstatic.com/fp/4...,299995,USD,3,2,1396,1396,Square Feet,sqft,{'comminglingCategoryIsRulesApplicable': True},12675,0.290978,Acres,"{'streetAddress': '3716 N 35th St', 'city': 'T...","{'location': [{'fullValue': '3716 N 35th St'},...",27.979225,-82.421130,FOR_SALE,SINGLE_FAMILY,/homedetails/3716-N-35th-St-Tampa-FL-33610/450...,"{'is_newHome': False, 'is_forAuction': False, ...",None,"{'mlsId': 'T3430083', 'mlsName': None, 'provid...",False,FL,False,None
4,2059031409,[{'url': 'https://photos.zillowstatic.com/fp/6...,419999,USD,3,2,1607,1607,Square Feet,sqft,{'comminglingCategoryIsRulesApplicable': True},5200,5200.000000,Square Feet,"{'streetAddress': '2803 N 33rd St', 'city': 'T...","{'location': [{'fullValue': '2803 N 33rd St'},...",27.968510,-82.423010,FOR_SALE,SINGLE_FAMILY,/homedetails/2803-N-33rd-St-Tampa-FL-33605/205...,"{'is_newHome': True, 'is_forAuction': False, '...",None,"{'mlsId': 'U8191744', 'mlsName': None, 'provid...",False,FL,False,BUILDER_SPEC
5,45082438,[{'url': 'https://photos.zillowstatic.com/fp/7...,339900,USD,3,2,1456,1456,Square Feet,sqft,{'comminglingCategoryIsRulesApplicable': True},11100,0.254821,Acres,"{'streetAddress': '1608 E Sitka St', 'city': '...",{'location': [{'fullValue': '1608 E Sitka St'}...,28.021452,-82.441956,FOR_SALE,SINGLE_FAMILY,/homedetails/1608-E-Sitka-St-Tampa-FL-33604/45...,"{'is_newHome': False, 'is_forAuction': False, ...",None,"{'mlsId': 'T3426989', 'mlsName': None, 'provid...",False,FL,False,None
6,45108845,[{'url': 'https://photos.zillowstatic.com/fp/6...,345000,USD,3,2,1187,1187,Square Feet,sqft,{'comminglingCategoryIsRulesApplicable': True},5300,5300.000000,Square Feet,"{'streetAddress': '1822 E 17th Ave', 'city': '...",{'location': [{'fullValue': '1822 E 17th Ave'}...,27.967752,-82.438150,FOR_SALE,SINGLE_FAMILY,/homedetails/1822-E-17th-Ave-Tampa-FL-33605/45...,"{'is_newHome': False, 'is_forAuction': False, ...",None,"{'mlsId': 'T3427190', 'mlsName': None, 'provid...",False,FL,False,None
7,45099406,[{'url': 'https://photos.zillowstatic.com/fp/f...,369000,USD,3,2,1858,1858,Square Feet,sqft,{'comminglingCategoryIsR

#### Tax History

In [23]:
# view tax history
df_tax_hist = pd.DataFrame(df_prop["taxHistory"].iloc[0])
df_tax_hist

,time,taxPaid,taxIncreaseRate,value,valueIncreaseRate
0,1614257548865,700.43,1.694998,35532,2.991014
1,1582635148865,259.90,0.029225,8903,0.099951
2,1551099148865,252.52,0.000000,8094,-0.509365
3,1519563148865,252.52,0.212290,16497,1.166667
4,1488027148865,208.30,0.450759,7614,0.252097
5,1456404748865,143.58,0.065055,6081,0.100036
6,1424868748865,134.81,0.212975,5528,0.100100
7,1393332748865,111.14,0.000000,5025,0.100044
8,1361796748865,111.14,0.129357,4568,0.000000
9,1330174348865,98.41,0.031660,4568,0.033952


#### Price History

In [24]:
# view price history
df_price_hist = pd.DataFrame(df_prop["priceHistory"].iloc[0])
df_price_hist

,date,time,price,pricePerSquareFoot,priceChangeRate,event,source,buyerAgent,sellerAgent,showCountyLink,postingIsRental,attributeSource
0,2023-02-04,1675468800000,350000,215,-0.041096,Price change,Stellar MLS / MFRMLS,None,None,False,False,"{'infoString1': 'T3424788', 'infoString2': 'St..."
1,2023-01-27,1674777600000,365000,224,0.383362,Listed for sale,Stellar MLS / MFRMLS,None,None,False,False,"{'infoString1': 'T3424788', 'infoString2': 'St..."
2,2023-01-02,1672617600000,2400,1,0.000000,Listing removed,HotPads,None,None,False,True,{'infoString1': '8da539eb3daa58a1532a386a012f2...
3,2022-12-05,1670198400000,2400,1,0.200000,Listed for rent,Zillow Rental Manager,None,None,False,True,"{'infoString1': None, 'infoString2': 'Zillow R..."
4,2021-12-16,1639612800000,2000,1,0.000000,Listing removed,Zillow Rental Network Premium,None,None,False,True,"{'infoString1': None, 'infoString2': 'Zillow R..."
5,2021-12-14,1639440000000,2000,1,-0.047619,Price change,Zillow Rental Network Premium,None,None,False,True,"{'infoString1': None, 'infoString2': 'Zillow R..."
6,2021-12-06,1638748800000,2100,1,-0.045455,Listed for rent,Zillow Rental Network Premium,None,None,False,True,"{'infoString1': None, 'infoString2': 'Zillow R..."
7,2021-10-15,1634256000000,2200,1,0.000000,Listing removed,Zillow Rental Network Premium,None,None,False,True,"{'infoString1': None, 'infoString2': 'Zillow R..."
8,2021-09-30,1632960000000,2200,1,0.000000,Listed for rent,Zillow Rental Network Premium,None,None,False,True,"{'infoString1': None, 'infoString2': 'Zillow R..."
9,2021-05-25,1621900800000,263850,162,0.000474,Sold,Stellar MLS / MFRMLS,{'photo': {'url': 'https://photos.zillowstatic...,None,False,False,"{'infoString1': 'T3270295', 'infoString2': 'St..."


### <font color="green">4. Locate ZPID by Address</font>

In [25]:
street = "12315 N Oakleaf Ave"
city = "Tampa"
state = "FL"
# zip_code = "33612" #[optional]

zpid_response = get_zpid(api_key, street, city, state, zip_code=None)

In [26]:
zpid_response.json()["is_success"]

True

In [27]:
# view response, top possible zpids
pd.DataFrame(zpid_response.json()["data"])

,zpid,url
0,44905318,https://www.zillow.com/homedetails/12315-N-Oak...
1,44905319,https://www.zillow.com/homedetails/12319-N-Oak...
2,44905317,https://www.zillow.com/homedetails/12311-N-Oak...
3,44905324,https://www.zillow.com/homedetails/12315-Woodl...
4,44905320,https://www.zillow.com/homedetails/1714-Round-...


In [28]:
print("Zpid:", zpid_response.json()["data"][0]["zpid"])

Zpid: 44905318


### <font color="green">5. FSBO (For Sale By Owner) Listings</font>

In [29]:
fsbo_url = "https://www.zillow.com/homes/fsbo/?searchQueryState=%7B%22pagination%22%3A%7B%7D%2C%22usersSearchTerm%22%3A%22Tampa%2C%20FL%22%2C%22mapBounds%22%3A%7B%22west%22%3A-82.92806590039064%2C%22east%22%3A-81.90908396679689%2C%22south%22%3A27.66429607182606%2C%22north%22%3A28.271955756480853%7D%2C%22category%22%3A%22cat2%22%2C%22customRegionId%22%3A%221561f63632X1-CRfbm1wqqt7k5h_1ghad5%22%2C%22savedSearchEnrollmentId%22%3A%22X1-SS9rmnd39e71et0000000000_8fet9%22%2C%22isMapVisible%22%3Atrue%2C%22filterState%22%3A%7B%22con%22%3A%7B%22value%22%3Afalse%7D%2C%22fore%22%3A%7B%22value%22%3Afalse%7D%2C%22apa%22%3A%7B%22value%22%3Afalse%7D%2C%22mf%22%3A%7B%22value%22%3Afalse%7D%2C%22ah%22%3A%7B%22value%22%3Atrue%7D%2C%22sort%22%3A%7B%22value%22%3A%22mostrecentchange%22%7D%2C%22auc%22%3A%7B%22value%22%3Afalse%7D%2C%22nc%22%3A%7B%22value%22%3Afalse%7D%2C%22land%22%3A%7B%22value%22%3Afalse%7D%2C%22tow%22%3A%7B%22value%22%3Afalse%7D%2C%22manu%22%3A%7B%22value%22%3Afalse%7D%2C%22cmsn%22%3A%7B%22value%22%3Afalse%7D%2C%22apco%22%3A%7B%22value%22%3Afalse%7D%2C%22fsba%22%3A%7B%22value%22%3Afalse%7D%7D%2C%22isListVisible%22%3Atrue%7D"

In [30]:
fspo_response = get_listings(api_key, fsbo_url)

In [31]:
# view count of properies returned in request
num_of_properties = fspo_response.json()["data"]["categoryTotals"]["cat2"]["totalResultCount"]
print("Count of properties:", num_of_properties)

Count of properties: 21


In [32]:
# view all listings
df_fsbo_listings = pd.json_normalize(fspo_response.json()["data"]["cat2"]["searchResults"]["mapResults"])
print("Number of rows:", len(df_fsbo_listings))
print("Number of columns:", len(df_fsbo_listings.columns))
df_fsbo_listings

Number of rows: 21
Number of columns: 73


,zpid,price,priceLabel,beds,baths,area,statusType,statusText,isFavorite,isUserClaimingOwner,isUserConfirmedClaim,imgSrc,hasImage,visited,listingType,shouldShowZestimateAsPrice,detailUrl,pgapt,sgapt,unreadNotification,has3DModel,hasVideo,isHomeRec,address,hasAdditionalAttributions,isFeaturedListing,availabilityDate,latLong.latitude,latLong.longitude,variableData.type,variableData.text,variableData.data.isRead,hdpData.homeInfo.zpid,hdpData.homeInfo.zipcode,hdpData.homeInfo.city,hdpData.homeInfo.state,hdpData.homeInfo.latitude,hdpData.homeInfo.longitude,hdpData.homeInfo.price,hdpData.homeInfo.datePriceChanged,hdpData.homeInfo.bathrooms,hdpData.homeInfo.bedrooms,hdpData.homeInfo.livingArea,hdpData.homeInfo.homeType,hdpData.homeInfo.homeStatus,hdpData.homeInfo.daysOnZillow,hdpData.homeInfo.isFeatured,hdpData.homeInfo.shouldHighlight,hdpData.homeInfo.zestimate,hdpData.homeInfo.rentZestimate,hdpData.homeInfo.listing_sub_type.is_FSBO,hdpData.homeInfo.listing_sub_type.is_openHouse,hdpData.homeInfo.openHouse,hdpData.homeInfo.priceReduction,hdpData.homeInfo.isUnmappable,hdpData.homeInfo.isPreforeclosureAuction,hdpData.homeInfo.homeStatusForHDP,hdpData.homeInfo.priceForHDP,hdpData.homeInfo.open_house_info.open_house_showing,hdpData.homeInfo.priceChange,hdpData.homeInfo.isNonOwnerOccupied,hdpData.homeInfo.isPremierBuilder,hdpData.homeInfo.isZillowOwned,hdpData.homeInfo.currency,hdpData.homeInfo.country,hdpData.homeInfo.taxAssessedValue,hdpData.homeInfo.lotAreaValue,hdpData.homeInfo.lotAreaUnit,hdpData.homeInfo.videoCount,streetViewMetadataURL,streetViewURL,variableData,hdpData.homeInfo.unit
0,44917636,"$429,000",$429K,5,2.0,1530,FOR_SALE,For sale by owner,False,False,False,https://photos.zillowstatic.com/fp/35dfcf6cd72...,True,False,,False,/homedetails/6002-N-Cameron-Ave-Tampa-FL-33614...,ForSale,For Sale By Owner,False,False,False,False,--,False,False,None,28.005276,-82.505920,SEARCH_NOTIFICATION_PRICE_REDUCTION,"$10,000 (02/23)",True,44917636,33614,Tampa,FL,28.005276,-82.505920,429000.0,1.677139e+12,2.0,5.0,1530.0,SINGLE_FAMILY,FOR_SALE,-1,False,False,213500.0,2334.0,True,True,Sat. 11am-2pm,"$10,000 (Feb 23)",False,False,FOR_SALE,429000.0,"[{'open_house_start': 1677351600000, 'open_hou...",-10000.0,True,False,False,USD,USA,137036.0,6534.000000,sqft,NaN,NaN,NaN,NaN,NaN
1,45107557,"$5,999,999",$6.00M,5,8.0,5971,FOR_SALE,For sale by owner,False,False,False,https://photos.zillowstatic.com/fp/9da4f347ea7...,True,False,,False,/homedetails/2114-W-Southview-Ave-Tampa-FL-336...,ForSale,For Sale By Owner,False,False,False,False,--,False,False,None,27.929918,-82.482790,SEARCH_NOTIFICATION_ADD_LISTING,Listed on 02/22/23,True,45107557,33606,Tampa,FL,27.929918,-82.482790,5999999.0,NaN,8.0,5.0,5971.0,SINGLE_FAMILY,FOR_SALE,-1,False,False,NaN,3597.0,True,NaN,NaN,NaN,False,False,FOR_SALE,5999999.0,NaN,NaN,True,False,False,USD,USA,375052.0,8651.000000,sqft,NaN,NaN,NaN,NaN,NaN
2,45085863,"$350,000",$350K,3,3.0,1438,FOR_SALE,For sale by owner,False,False,False,https://photos.zillowstatic.com/fp/95f2f9aac8b...,True,False,,False,/homedetails/3906-E-Louisiana-Ave-Tampa-FL-336...,ForSale,For Sale By Owner,False,False,True,False,--,False,False,None,27.989887,-82.415560,SEARCH_NOTIFICATION_ADD_LISTING,Listed on 02/21/23,True,45085863,33610,Tampa,FL,27.989887,-82.415560,350000.0,NaN,3.0,3.0,1438.0,SINGLE_FAMILY,FOR_SALE,-1,False,False,284300.0,2075.0,True,NaN,NaN,NaN,False,False,FOR_SALE,350000.0,NaN,NaN,True,False,False,USD,USA,184696.0,8258.000000,sqft,1.0,NaN,NaN,NaN,NaN
3,45096441,"$485,000",$485K,2,2.0,1200,FOR_SALE,For sale by owner,False,False,False,https://photos.zillowstatic.com/fp/37eb64ad8da...,True,False,,False,/homedetails/43-Hamilton-Heath-Dr-Tampa-FL-336...,ForSale,For Sale By Owner,False,False,False,False,--,False,False,None,28.018715,-82.449290,SEARCH_NOTIFICATION_ADD_LISTING,Listed on 02/21/23,True,45096441,33604,Tampa,FL,28.018715,-82.449290,485000.0,NaN,2.0,2.0,1200.0,SINGLE_FAMILY,FOR_SALE,-1,False,False,339800.0,1900.0,True,NaN,NaN,NaN,False,

#### <font color="purple">Get Owner Phone Number</font>

In [33]:
# select a sample fspo property from the dataframe
fsbo_single_listing = "44917636" # zpid 

In [34]:
# get property detail
fsbo_prop_detail_response = get_property_detail(api_key, fsbo_single_listing)

# view property details
df_fsbo_prop = pd.json_normalize(fsbo_prop_detail_response.json()['data'])
print("Number of rows:", len(df_fsbo_prop))
print("Number of columns:", len(df_fsbo_prop.columns))
df_fsbo_prop

Number of rows: 1
Number of columns: 607


,listingDataSource,zpid,city,state,homeStatus,isListingClaimedByCurrentSignedInUser,isCurrentSignedInAgentResponsible,bedrooms,bathrooms,price,yearBuilt,streetAddress,zipcode,isCurrentSignedInUserVerifiedOwner,regionString,propertyUpdatePageLink,moveHomeMapLocationLink,propertyEventLogLink,editPropertyHistorylink,isRentalListingOffMarket,hdpUrl,nearbyCities,nearbyNeighborhoods,country,nearbyZipcodes,abbreviatedAddress,building,isUndisclosedAddress,boroughId,providerListingID,boroughSearchUrl,communityUrl,isPremierBuilder,isZillowOwned,homeType,currency,listPriceLow,livingArea,livingAreaValue,zestimate,newConstructionType,zestimateLowPercent,zestimateHighPercent,rentZestimate,restimateLowPercent,restimateHighPercent,schools,homeValues,nearbyHomes,countyFIPS,parcelId,taxHistory,priceHistory,comps,description,whatILove,contingentListingType,timeOnZillow,pageViewCount,favoriteCount,daysOnZillow,latitude,longitude,openHouseSchedule,desktopWebHdpImageLink,brokerageName,timeZone,pals,listingAccountUserId,homeInsights,sellingSoon,isIncomeRestricted,brokerId,ssid,monthlyHoaFee,propertyTaxRate,hiResImageLink,hdpTypeDimension,mlsid,propertyTypeDimension,mediumImageLink,enhancedBrokerImageUrl,responsivePhotos,buildingId,virtualTourUrl,hasApprovedThirdPartyVirtualTourUrl,photoCount,livingAreaUnits,lotSize,lotAreaValue,lotAreaUnits,postingProductType,marketingName,richMedia,cityId,stateId,zipPlusFour,numberOfUnitsTotal,foreclosureDefaultFilingDate,foreclosureAuctionFilingDate,foreclosureLoanDate,foreclosureLoanOriginator,foreclosureLoanAmount,foreclosurePriorSaleDate,foreclosurePriorSaleAmount,foreclosureBalanceReportingDate,foreclosurePastDueBalance,foreclosureUnpaidBalance,foreclosureAuctionTime,foreclosureAuctionDescription,foreclosureAuctionCity,foreclosureAuctionLocation,foreclosureDate,foreclosureAmount,foreclosingBank,foreclosureJudicialType,datePostedString,foreclosureMoreInfo,hasBadGeocode,streetViewMetadataUrlMediaWallLatLong,streetViewMetadataUrlMediaWallAddress,streetViewTileImageUrlMediumLatLong,streetViewTileImageUrlMediumAddress,streetViewServiceUrl,postingUrl,hasPublicVideo,primaryPublicVideo,richMediaVideos,photos,tourViewCount,listingAccount,listingFeedID,livingAreaUnitsShort,priceChange,priceChangeDate,priceChangeDateString,hideZestimate,comingSoonOnMarketDate,isPreforeclosureAuction,lastSoldPrice,isHousingConnector,responsivePhotosOriginalRatio,streetViewMetadataUrlMapLightboxAddress,thumb,isRecentStatusChange,isNonOwnerOccupied,county,isFeatured,rentalApplicationsAcceptedType,listingTypeDimension,featuredListingTypeDimension,brokerIdDimension,keystoneHomeStatus,pageUrlFragment,isRentalsLeadCapMet,isPaidMultiFamilyBrokerId,address.streetAddress,address.city,address.state,address.zipcode,address.neighborhood,address.community,address.subdivision,listing_sub_type.is_newHome,listing_sub_type.is_FSBO,listing_sub_type.is_FSBA,listing_sub_type.is_foreclosure,listing_sub_type.is_bankOwned,listing_sub_type.is_forAuction,listing_sub_type.is_comingSoon,listing_sub_type.is_pending,listing_sub_type.is_openHouse,citySearchUrl.text,citySearchUrl.path,zipcodeSearchUrl.path,apartmentsForRentInZipcodeSearchUrl.path,housesForRentInZipcodeSearchUrl.path,neighborhoodRegion.name,neighborhoodSearchUrl.path,stateSearchUrl.path,countySearchUrl.text,countySearchUrl.path,adTargets.aamgnrc1,adTargets.bd,adTargets.city,adTargets.proptp,adTargets.pid,adTargets.lot,adTargets.zestibuck,adTargets.listtp,adTargets.sqftrange,adTargets.price,adTargets.oh,adTargets.sqft,adTargets.state,adTargets.mlong,adTargets.cnty,adTargets.prange,adTargets.zip,adTargets.mlat,adTargets.zusr,adTargets.price_band,adTargets.yrblt,adTargets.zestimate,adTargets.premieragent,adTargets.dma,adTargets.guid,adTargets.ssid,resoFacts.accessibilityFeatures,resoFacts.additionalFeeInfo,resoFacts.associationFee,resoFacts.associationAmenities,resoFacts.associationFee2,resoFacts.associationFeeIncludes,resoFacts.associationName,resoFacts.associationName2,resoFacts.associationPhone,resoFact

In [35]:
# features
# phone number
df_fsbo_prop["phone_number"] = df_fsbo_prop.apply(lambda x: 
  x["contactFormRenderData.data.agent_module.phone.areacode"] + '-' +
  x["contactFormRenderData.data.agent_module.phone.prefix"] + '-' +
  x["contactFormRenderData.data.agent_module.phone.number"], axis=1)
# full address
df_fsbo_prop["full_address"] = df_fsbo_prop.apply(lambda x: 
  x["streetAddress"] + ', ' +
  x["city"] + ', ' +
  x["state"] + ' ' + x["zipcode"], axis=1)

print("The phone number for property address {} is {}".format(
    list(df_fsbo_prop["full_address"])[0],
    list(df_fsbo_prop["phone_number"])[0]))

The phone number for property address 6002 N Cameron Ave, Tampa, FL 33614 is 813-789-0858


### <font color="green">6. Canada Listings</font>
*Coming soon*

# End Notebook